In [2]:
import turicreate as tc
import json
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
dtype_dict = {'name':str, 'review':str, 'rating':float}
products = tc.SFrame('m_bfaa91c17752f745.frame_idx')
products_pd = pd.read_csv('amazon_baby.csv', dtype=dtype_dict)
products = products.fillna('review','')
products_pd = products_pd.fillna({'review':''})

In [4]:
products_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183531 entries, 0 to 183530
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   name    183213 non-null  object 
 1   review  183531 non-null  object 
 2   rating  183531 non-null  float64
dtypes: float64(1), object(2)
memory usage: 4.2+ MB


In [5]:
import string
def remove_punctuation(text):
    translator = str.maketrans('','',string.punctuation)
    return text.translate(translator) 
#products['review_clean'] = products['review'].apply(remove_punctuation)

In [6]:
products['review_clean'] = products['review'].apply(remove_punctuation)
products_pd['review_clean'] = products_pd['review'].apply(remove_punctuation)

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 32)"

Note that increasing the degree of parallelism also increases the memory footprint.

In [ ]:
print(products['review_clean'][2])
print(products['review'][2])

In [7]:
products = products[products['rating'] != 3]
products_pd = products_pd[products_pd['rating'] != 3]

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products_pd['sentiment'] = products_pd['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [9]:
with open('module-2-assignment-test-idx.json') as test_data_file:    
    test_data_idx = json.load(test_data_file)
with open('module-2-assignment-train-idx.json') as train_data_file:    
    train_data_idx = json.load(train_data_file)

train_data = products_pd.iloc[train_data_idx]
test_data = products_pd.iloc[test_data_idx]

In [ ]:
"""train_data, test_data = products.random_split(.8, seed=1)
train_data_pd, test_data_pd, y_train_pd, y_test_pd = train_test_split(
    products_pd[['name', 'review', 'rating','review_clean']], 
    products_pd['sentiment'], 
    test_size=0.2, random_state=1)"""

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
test_matrix = vectorizer.transform(test_data['review_clean'])

In [ ]:
"""

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])
train_matrix_pd = vectorizer.fit_transform(train_data_pd['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix_pd = vectorizer.transform(test_data_pd['review_clean'])

In [11]:
sentiment_model = linear_model.LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
np.sum(sentiment_model.coef_ >= 0)

90357

In [ ]:
np.count_nonzero(sentiment_model.intercept_ >= 0) + np.count_nonzero(sentiment_model.coef_ >= 0) 

In [13]:
sample_test_data = test_data.iloc[10:13]
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
print(sentiment_model.classes_)
print(sentiment_model.predict_proba(sample_test_matrix))


[-1  1]
[[7.52087610e-03 9.92479124e-01]
 [9.57188195e-01 4.28118053e-02]
 [9.99958420e-01 4.15795848e-05]]


In [ ]:
sample_test_data[0]['review']

In [ ]:
sample_test_data[1]['review']

In [ ]:
sample_test_data[2]['review']

In [14]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print (scores)

[  4.88252334  -3.10718613 -10.08785968]


In [15]:
predicted_test_sample = sentiment_model.predict(sample_test_matrix)
print(predicted_test_sample)

[ 1 -1 -1]


In [16]:
probability_test_sample = sentiment_model.predict_proba(sample_test_matrix)
print(probability_test_sample)

[[7.52087610e-03 9.92479124e-01]
 [9.57188195e-01 4.28118053e-02]
 [9.99958420e-01 4.15795848e-05]]


In [17]:
lowest = np.argmin(probability_test_sample, axis = 0)
print(lowest[1] + 1)

3


In [18]:
probability_test_data = sentiment_model.predict_proba(test_matrix)
predicted_test_sample = sentiment_model.predict(test_matrix)
test_data['positive probability'] = [x[1] for x in np.asarray(probability_test_data)]
top_positive_reviews = test_data.sort_values(by='positive probability',ascending=False)[:20]

In [37]:
top_positive_reviews['name'].sort_values()

87017       Baby Einstein Around The World Discovery Center
147996    Baby Jogger City Mini GT Double Stroller, Shad...
147949    Baby Jogger City Mini GT Single Stroller, Shad...
133651                    Britax 2012 B-Agile Stroller, Red
166827    Britax Boulevard 70-G3 Convertible Car Seat Se...
168081    Buttons Cloth Diaper Cover - One Size - 8 Colo...
140816           Diono RadianRXT Convertible Car Seat, Plum
66059          Evenflo 6 Pack Classic Glass Bottle, 4-Ounce
52631     Evenflo X Sport Plus Convenience Stroller - Ch...
114796    Fisher-Price Cradle 'N Swing,  My Little Snuga...
168697    Graco FastAction Fold Jogger Click Connect Str...
137034           Graco Pack 'n Play Element Playard - Flint
165593    Ikea 36 Pcs Kalas Kids Plastic BPA Free Flatwa...
100166    Infantino Wrap and Tie Baby Carrier, Black Blu...
180646        Mamas &amp; Papas 2014 Urbo2 Stroller - Black
50315            P'Kolino Silly Soft Seating in Tias, Green
119182    Roan Rocco Classic Pram Stroll

In [20]:
test_data['negative probability'] = [x[0] for x in np.asarray(probability_test_data)]
top_negative_reviews = test_data.sort_values(by='negative probability',ascending=False)[:20]

In [21]:
top_negative_reviews['name']

94560     The First Years True Choice P400 Premium Digit...
16042           Fisher-Price Ocean Wonders Aquarium Bouncer
120209    Levana Safe N'See Digital Video Baby Monitor w...
155287    VTech Communications Safe &amp; Sounds Full Co...
48694     Adiri BPA Free Natural Nurser Ultimate Bottle ...
95420          One Step Ahead Hide-Away Extra Long Bed Rail
53207                   Safety 1st High-Def Digital Monitor
176046         Baby Trend Inertia Infant Car Seat - Horizon
167249    Samsung SEW-3037W Wireless Pan Tilt Video Baby...
81332                 Cloth Diaper Sprayer--styles may vary
94389                  Snuza Portable Baby Movement Monitor
59546                Ellaroo Mei Tai Baby Carrier - Hershey
31741                Regalo My Cot Portable Bed, Royal Blue
76000            Peg-Perego Tatamia High Chair, White Latte
77072        Safety 1st Exchangeable Tip 3 in 1 Thermometer
96572      Baby Jogger Summit XC Double Stroller, Red/Black
1116                  Safety 1st Deluxe 

In [22]:
def get_classification_accuracy(model, data, true_labels):
    pred_y = model.predict(data)
    correct = np.sum(pred_y==true_labels)
    accuracy = round(correct/len(true_labels), 2)
    return accuracy

accuracy_prob = get_classification_accuracy(sentiment_model, test_matrix, test_data["sentiment"])
print(accuracy_prob)

0.93


In [23]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [26]:
test_matrix_word_subset.toarray()

array([[0, 0, 1, ..., 0, 1, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [27]:
simple_model = linear_model.LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
simple_model_coef_table = sframe.SFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})

In [28]:
simple_model_coefficient = pd.DataFrame(
    {'word':significant_words,'simple_model_coefficient':simple_model.coef_.flatten()}).sort_values(
    ['simple_model_coefficient'], ascending=False).reset_index(drop=True)

len(simple_model_coefficient[simple_model_coefficient["simple_model_coefficient"]>0])

10

In [31]:
accuracy_simple = get_classification_accuracy(simple_model, train_matrix_word_subset, train_data['sentiment'])
accuracy_complex = get_classification_accuracy(sentiment_model, train_matrix, train_data['sentiment'])
print(accuracy_simple)
print(accuracy_complex)

0.87
0.95


In [32]:
print("Sentiment Model: ",get_classification_accuracy(sentiment_model,test_matrix,test_data["sentiment"]))
print("Simple Model: ",get_classification_accuracy(simple_model,test_matrix_word_subset,test_data["sentiment"]))

Sentiment Model:  0.93
Simple Model:  0.87


In [33]:
freq=pd.crosstab(test_data["sentiment"], columns=["count"]).reset_index()
freq

col_0,sentiment,count
0,-1,5241
1,1,28095


In [34]:
baseline_model=round(freq[freq["sentiment"]==1]["count"].values[0]/freq["count"].sum(),2)
print("Baseline Model: ", baseline_model)

Baseline Model:  0.84
